# Анализ базы данных сервиса чтения книг по подписке

***Цель исследования*** - проанализировать данные в базе данных сервиса, чтобы в дальнейшем выдвинуть ценностное предложение

***Описание исследования***

После покупки сервиса для чтения книг по подписке первая задача — проанализировать базу данных. Данные помогут сформулировать ценностное предложение для нового продукта.

***Ход исследования***

Необходимо произвести обзор данных и ответить на вопросы:

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Подключение к базе и обзор данных

Импортируем библиотеки, которые потребуются нам для работы:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

Устанавливаем необходимые параметры:

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

Сохраняем коннектор:

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Добавляем функцию для чления запросов:

In [4]:
# функция для чтения sql запроса
def select(sql):
  return pd.read_sql(sql,engine)

Откроем каждую из таблиц и посмотрим ее содержимое:

In [21]:
# читаем таблицу books
books = '''
SELECT 
    *
FROM books
'''

books_tab = select(books)
books_tab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [23]:
books_tab.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [24]:
# читаем таблицу authors
authors = '''
SELECT 
    *
FROM authors
'''

authors_tab = select(authors)
authors_tab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [25]:
authors_tab.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [26]:
# читаем таблицу ratings
ratings = '''
SELECT 
    *
FROM ratings
'''

ratings_tab = select(ratings)
ratings_tab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [27]:
ratings_tab.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [29]:
# читаем таблицу reviews
reviews = '''
SELECT 
    *
FROM reviews
'''

reviews_tab = select(reviews)
reviews_tab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [30]:
reviews_tab.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [31]:
# читаем таблицу publishers
publishers = '''
SELECT 
    *
FROM publishers
'''

publishers_tab = select(publishers)
publishers_tab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [32]:
publishers_tab.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Содержание всех таблиц соответствует заявленному. Известно, что поля в таблицах обозначены вот так:

Таблица books
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица ratings
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews
Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

Схема данных выглядит так:
![](https://pictures.s3.yandex.net/resources/scheme_1589269096.png) 


Приступим к решению задач. 

## Решение задач

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [10]:
books_count = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''

select(books_count)

,count
0,819


Теперь нам известно, что после 1 января 2000 года было выпущено 819 книг. 

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [11]:
books_details = '''
WITH
a AS (SELECT book_id, COUNT(review_id) AS c_rev
      FROM reviews
      GROUP BY book_id),
b AS (SELECT book_id, AVG(rating) AS avg_rat
      FROM ratings
      GROUP BY book_id)      


SELECT books.book_id,
books.title, 
c_rev,
avg_rat
FROM books 
left JOIN a ON books.book_id = a.book_id
left JOIN b ON books.book_id = b.book_id
ORDER BY avg_rat DESC, c_rev DESC
'''

select(books_details)

,book_id,title,c_rev,avg_rat
0,672,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.00
1,17,A Dirty Job (Grim Reaper #1),4.0,5.00
2,553,School's Out—Forever (Maximum Ride #2),3.0,5.00
3,444,Moneyball: The Art of Winning an Unfair Game,3.0,5.00
4,516,Plum Island (John Corey #1),2.0,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,202,Drowning Ruth,3.0,2.00
997,371,Junky,2.0,2.00
998,316,His Excellency: George Washington,2.0,2.00


Всего у нас 1000 книг со средними оценками пользователей от 5 до 1,5. При этом общее количество отзывов может быть разным. Например, есть книги имеющие рейтинг 5, но не имеющие отзывов совсем. В тоже время мы видим и обратную историю. 

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [12]:
books_publishers = '''
SELECT publisher,
COUNT(book_id) AS coun_b
FROM books AS b
left JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY coun_b DESC
LIMIT 1
'''

select(books_publishers)

,publisher,coun_b
0,Penguin Books,42


Больше всего книг (42) выпустило издательство Penguin Books. Все книги больше 50 страниц. 

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [20]:
books_rating = '''    
SELECT au.author_id, au.author, AVG(a.avg_rat) AS avg_rat, SUM(a.count_rat) as count_rat
FROM (SELECT book_id, 
      AVG(rating) as avg_rat,
      COUNT(rating) AS count_rat
      FROM ratings 
      GROUP BY book_id
      HAVING COUNT(rating) >= 50) AS a
left JOIN books AS b ON a.book_id = b.book_id
left JOIN authors AS au ON b.author_id = au.author_id
GROUP BY au.author_id, au.author
ORDER BY avg_rat DESC
LIMIT 1
'''

select(books_rating)

,author_id,author,avg_rat,count_rat
0,236,J.K. Rowling/Mary GrandPré,4.283844,310.0


При условии, что нам нужно было учитывать только авторов с количеством оценок более 50, то самая высокая оценка книг получилась у Джоан Роулин.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [ ]:
books_reviews = '''
WITH
a AS (SELECT username,
COUNT(review_id) AS count_rev
FROM reviews
WHERE username IN (SELECT username
                   FROM (SELECT username, 
                         COUNT(rating) AS count_rat
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating) > 50) AS b)
GROUP BY username)


SELECT AVG(count_rev)
FROM a
'''

select(books_reviews)

Те пользователи, которые ставят больше всего оценок, показывают и высокие значения по количеству обзоров. В среднем у них получилось 24 обзора - это довольно много. 

## Вывод

Мы произвели необходимые выгрузки данных. У нас получилось:
- всего книг выпущенных после 1 января 2000 года у нас 819
- всего книг у нас 1000. Книги имеют рейтинг от 1,5 до 5. При этом количество отзывов не всегда зависит от рейтинга: книги с высоким рейтингом могут не иметь отзывов, книги с низким рейтингом могут иметь достаточно большое количество отзывов
- больше всего книг выпускает издательство Penguin Books	
- самым популярным автором (по рейтингу и количеству оценок) является J.K. Rowling/Mary GrandPré	
- те пользователи, которые оставляют достаточно много оценок, так же часто пишут обзоры. В среднем те, кто оставил больше 50 оценок, пишут 24 отзыва